### 1. Carrega o arquivo com as sentenças, features e classificação

In [1]:
import pandas as pd

data_frame = pd.read_csv('dataset_sentencas_processadas_400.csv')

# sentenca_original = sentenças obtidas do trabalho de Gabriela
# sentenca_processada1 = remoção das anotações feitas por Gabriela
# sentenca_processada2 = sem pontuações
# sentenca_processada3_1 = sem acentos
# sentenca_processada3_2 = sem stopwords
# sentenca_processada3_3 = letras minúsculas
# sentenca_processada3_4 = apenas as raizes das palavras (stemmer)

In [2]:
data_frame.head(1)

,classificacao,sentenca,sentenca_processada1,sentenca_processada2,sentenca_processada3_1,sentenca_processada3_2,sentenca_processada3_3,sentenca_processada3_4
0,1.0,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca,Parece estar no Reino da Dinamarca,Parece estar Reino Dinamarca,parece estar reino dinamarca,parec est rein dinamarc


In [3]:
print(data_frame['classificacao'].value_counts())

0.0    200
1.0    200
Name: classificacao, dtype: int64


### 2. TF-IDF (Term Frequency–Inverse Document Frequency)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vetorizar = TfidfVectorizer(lowercase=False)

tfidf = vetorizar.fit_transform(data_frame['sentenca_processada3_4'].values.astype('U'))

vocabulario = vetorizar.get_feature_names()

In [5]:
print(tfidf.shape)

(400, 1389)


#### 2.1. Utiliza conjunto de treino/teste (80/20% e 90/10%) para cálculo Precisão, Acurácia, F1-Score e Recall

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(tfidf, data_frame['classificacao'], random_state = i, test_size = 0.1)

    # Treina o modelo usando o conjunto de dados de treino:
    classificador = tree.DecisionTreeClassifier()
    classificador.fit(treino, classe_treino)

    # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
    previsao = classificador.predict_proba(teste)

    # transforma as saídas classificadas de acordo com um limiar:
    previsao_bool = previsao[:,1] >= 0.5

    # transforma as saídas classificadas (booleanas) em valores inteiros:
    previsao_int = previsao_bool.astype(np.int)
    
    f1_score = metrics.f1_score(classe_teste, previsao_int)
    accuracy = metrics.accuracy_score(classe_teste, previsao_int)
    precision = metrics.precision_score(classe_teste, previsao_int)
    recall = metrics.recall_score(classe_teste, previsao_int)
    
    if(max_f1 < f1_score): 
        max_f1 = f1_score
        max_i_f1 = i
        
    if(max_precision < precision): 
        max_precision = precision
        max_i_precision = i

    # Apresenta os resultados de avaliação do algoritmo de classificação
    print(i, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0.48 	 0.33 	 0.62 	 0.43
1 	 0.55 	 0.37 	 0.54 	 0.44
2 	 0.42 	 0.32 	 0.47 	 0.38
3 	 0.48 	 0.5 	 0.52 	 0.51
4 	 0.48 	 0.29 	 0.64 	 0.4
5 	 0.62 	 0.39 	 0.64 	 0.48
6 	 0.57 	 0.48 	 0.69 	 0.56
7 	 0.6 	 0.42 	 0.62 	 0.5
8 	 0.57 	 0.32 	 0.6 	 0.41
9 	 0.48 	 0.36 	 0.53 	 0.43
10 	 0.5 	 0.33 	 0.54 	 0.41
11 	 0.45 	 0.25 	 0.29 	 0.27
12 	 0.55 	 0.44 	 0.44 	 0.44
13 	 0.48 	 0.35 	 0.47 	 0.4
14 	 0.48 	 0.26 	 0.6 	 0.36
15 	 0.5 	 0.41 	 0.41 	 0.41
16 	 0.55 	 0.47 	 0.53 	 0.5
17 	 0.48 	 0.38 	 0.5 	 0.43
18 	 0.6 	 0.5 	 0.62 	 0.56
19 	 0.62 	 0.45 	 0.69 	 0.55
20 	 0.52 	 0.38 	 0.69 	 0.49
21 	 0.62 	 0.52 	 0.81 	 0.63
22 	 0.5 	 0.29 	 0.38 	 0.33
23 	 0.55 	 0.45 	 0.56 	 0.5
24 	 0.52 	 0.44 	 0.69 	 0.54
25 	 0.75 	 0.53 	 0.73 	 0.62
26 	 0.57 	 0.42 	 0.57 	 0.48
27 	 0.57 	 0.57 	 0.6 	 0.59
28 	 0.45 	 0.26 	 0.38 	 0.31
29 	 0.62 	 0.42 	 0.67 	 0.52
30 	 0.6 	 0.42 	 0.83 	 0.56
31 	 0.57 	 0.5 	 0.47 	 0.48
32 	 0.48 	 0.42 	 0.44 	 0.43
33 	 

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(tfidf, data_frame['classificacao'], random_state = i, test_size = 0.1)

    # Treina o modelo usando o conjunto de dados de treino:
    classificador = LogisticRegression()
    classificador.fit(treino, classe_treino)

    # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
    previsao = classificador.predict_proba(teste)

    # transforma as saídas classificadas de acordo com um limiar:
    previsao_bool = previsao[:,1] >= 0.5

    # transforma as saídas classificadas (booleanas) em valores inteiros:
    previsao_int = previsao_bool.astype(np.int)
    
    f1_score = metrics.f1_score(classe_teste, previsao_int)
    accuracy = metrics.accuracy_score(classe_teste, previsao_int)
    precision = metrics.precision_score(classe_teste, previsao_int)
    recall = metrics.recall_score(classe_teste, previsao_int)
    
    if(max_f1 < f1_score): 
        max_f1 = f1_score
        max_i_f1 = i
        
    if(max_precision < precision): 
        max_precision = precision
        max_i_precision = i

    # Apresenta os resultados de avaliação do algoritmo de classificação
    print(i, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0.57 	 0.58 	 0.67 	 0.62
1 	 0.55 	 0.42 	 0.53 	 0.47
2 	 0.55 	 0.55 	 0.6 	 0.57
3 	 0.52 	 0.5 	 0.58 	 0.54
4 	 0.5 	 0.58 	 0.58 	 0.58
5 	 0.45 	 0.39 	 0.39 	 0.39
6 	 0.55 	 0.57 	 0.62 	 0.59
7 	 0.52 	 0.47 	 0.5 	 0.49
8 	 0.5 	 0.53 	 0.48 	 0.5
9 	 0.65 	 0.73 	 0.67 	 0.7
10 	 0.52 	 0.52 	 0.55 	 0.54
11 	 0.62 	 0.62 	 0.53 	 0.57
12 	 0.48 	 0.62 	 0.4 	 0.49
13 	 0.52 	 0.7 	 0.52 	 0.6
14 	 0.6 	 0.52 	 0.71 	 0.6
15 	 0.5 	 0.76 	 0.45 	 0.57
16 	 0.52 	 0.68 	 0.5 	 0.58
17 	 0.6 	 0.62 	 0.62 	 0.62
18 	 0.6 	 0.55 	 0.61 	 0.58
19 	 0.65 	 0.6 	 0.67 	 0.63
20 	 0.55 	 0.54 	 0.65 	 0.59
21 	 0.48 	 0.44 	 0.61 	 0.51
22 	 0.4 	 0.41 	 0.33 	 0.37
23 	 0.52 	 0.55 	 0.52 	 0.54
24 	 0.62 	 0.6 	 0.75 	 0.67
25 	 0.62 	 0.87 	 0.5 	 0.63
26 	 0.5 	 0.47 	 0.47 	 0.47
27 	 0.6 	 0.57 	 0.63 	 0.6
28 	 0.4 	 0.42 	 0.38 	 0.4
29 	 0.42 	 0.42 	 0.4 	 0.41
30 	 0.55 	 0.54 	 0.65 	 0.59
31 	 0.62 	 0.88 	 0.52 	 0.65
32 	 0.55 	 0.63 	 0.52 	 0.57
33 	 0.5 	 0.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(tfidf, data_frame['classificacao'], random_state = i, test_size = 0.1)

    # Treina o modelo usando o conjunto de dados de treino:
    classificador = svm.SVC(gamma='auto', C=1.0, kernel='linear', probability=True)
    classificador.fit(treino, classe_treino)

    # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
    previsao = classificador.predict_proba(teste)

    # transforma as saídas classificadas de acordo com um limiar:
    previsao_bool = previsao[:,1] >= 0.5

    # transforma as saídas classificadas (booleanas) em valores inteiros:
    previsao_int = previsao_bool.astype(np.int)
    
    f1_score = metrics.f1_score(classe_teste, previsao_int)
    accuracy = metrics.accuracy_score(classe_teste, previsao_int)
    precision = metrics.precision_score(classe_teste, previsao_int)
    recall = metrics.recall_score(classe_teste, previsao_int)
    
    if(max_f1 < f1_score): 
        max_f1 = f1_score
        max_i_f1 = i
        
    if(max_precision < precision): 
        max_precision = precision
        max_i_precision = i

    # Apresenta os resultados de avaliação do algoritmo de classificação
    print(i, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0.52 	 0.29 	 0.78 	 0.42
1 	 0.65 	 0.58 	 0.65 	 0.61
2 	 0.6 	 0.55 	 0.67 	 0.6
3 	 0.52 	 0.5 	 0.58 	 0.54
4 	 0.57 	 0.62 	 0.65 	 0.64
5 	 0.45 	 0.56 	 0.42 	 0.48
6 	 0.42 	 0.52 	 0.5 	 0.51
7 	 0.52 	 0.47 	 0.5 	 0.49
8 	 0.6 	 0.84 	 0.55 	 0.67
9 	 0.65 	 0.77 	 0.65 	 0.71
10 	 0.5 	 0.62 	 0.52 	 0.57
11 	 0.6 	 0.62 	 0.5 	 0.56
12 	 0.5 	 0.69 	 0.42 	 0.52
13 	 0.48 	 0.65 	 0.48 	 0.55
14 	 0.57 	 0.57 	 0.65 	 0.6
15 	 0.48 	 0.76 	 0.43 	 0.55
16 	 0.52 	 0.74 	 0.5 	 0.6
17 	 0.6 	 0.62 	 0.62 	 0.62
18 	 0.55 	 0.55 	 0.55 	 0.55
19 	 0.32 	 0.55 	 0.38 	 0.45
20 	 0.52 	 0.5 	 0.63 	 0.56
21 	 0.48 	 0.4 	 0.62 	 0.49
22 	 0.45 	 0.59 	 0.4 	 0.48
23 	 0.5 	 0.6 	 0.5 	 0.55
24 	 0.62 	 0.64 	 0.73 	 0.68
25 	 0.52 	 1.0 	 0.44 	 0.61
26 	 0.6 	 0.68 	 0.57 	 0.62
27 	 0.6 	 0.57 	 0.63 	 0.6
28 	 0.4 	 0.42 	 0.38 	 0.4
29 	 0.42 	 0.47 	 0.41 	 0.44
30 	 0.57 	 0.62 	 0.65 	 0.64
31 	 0.4 	 1.0 	 0.4 	 0.57
32 	 0.5 	 0.84 	 0.48 	 0.62
33 	 0.45 	 0.62 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(50):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(tfidf, data_frame['classificacao'], random_state = i, test_size = 0.1)

    max_f1_j = 0
    max_precision_j = 0
    max_j_f1 = 0
    max_j_precision = 0

    for j in range(50):
        # Treina o modelo usando o conjunto de dados de treino:
        classificador = MLPClassifier(activation='relu', solver='adam', max_iter=10000, alpha=1e-10, random_state=j)
        classificador.fit(treino, classe_treino)

        # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
        previsao = classificador.predict_proba(teste)

        # transforma as saídas classificadas de acordo com um limiar:
        previsao_bool = previsao[:,1] >= 0.5

        # transforma as saídas classificadas (booleanas) em valores inteiros:
        previsao_int = previsao_bool.astype(np.int)

        f1_score = metrics.f1_score(classe_teste, previsao_int)
        accuracy = metrics.accuracy_score(classe_teste, previsao_int)
        precision = metrics.precision_score(classe_teste, previsao_int)
        recall = metrics.recall_score(classe_teste, previsao_int)

        if(max_f1_j < f1_score): 
            max_f1_j = f1_score
            max_j_f1 = j

        if(max_precision_j < precision): 
            max_precision_j = precision
            max_j_precision = j

        # Apresenta os resultados de avaliação do algoritmo de classificação
        print(i, '\t', j, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))

    print(max_j_precision, round(max_precision_j, 2), max_j_f1, round(max_f1_j, 2))
    
    if(max_f1 < max_f1_j): 
        max_f1 = max_f1_j
        max_i_f1 = i

    if(max_precision < max_precision_j): 
        max_precision = max_precision_j
        max_i_precision = i
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0 	 0.6 	 0.62 	 0.68 	 0.65
0 	 1 	 0.57 	 0.58 	 0.67 	 0.62
0 	 2 	 0.57 	 0.58 	 0.67 	 0.62
0 	 3 	 0.57 	 0.58 	 0.67 	 0.62
0 	 4 	 0.6 	 0.58 	 0.7 	 0.64
0 	 5 	 0.6 	 0.58 	 0.7 	 0.64
0 	 6 	 0.57 	 0.58 	 0.67 	 0.62
0 	 7 	 0.52 	 0.5 	 0.63 	 0.56
0 	 8 	 0.55 	 0.5 	 0.67 	 0.57
0 	 9 	 0.57 	 0.58 	 0.67 	 0.62
0 	 10 	 0.6 	 0.62 	 0.68 	 0.65
0 	 11 	 0.57 	 0.58 	 0.67 	 0.62
0 	 12 	 0.57 	 0.58 	 0.67 	 0.62
0 	 13 	 0.57 	 0.58 	 0.67 	 0.62
0 	 14 	 0.55 	 0.54 	 0.65 	 0.59
0 	 15 	 0.55 	 0.54 	 0.65 	 0.59
0 	 16 	 0.57 	 0.58 	 0.67 	 0.62
0 	 17 	 0.62 	 0.62 	 0.71 	 0.67
0 	 18 	 0.55 	 0.54 	 0.65 	 0.59
0 	 19 	 0.52 	 0.5 	 0.63 	 0.56
0 	 20 	 0.57 	 0.58 	 0.67 	 0.62
0 	 21 	 0.55 	 0.54 	 0.65 	 0.59
0 	 22 	 0.6 	 0.62 	 0.68 	 0.65
0 	 23 	 0.55 	 0.54 	 0.65 	 0.59
0 	 24 	 0.57 	 0.58 	 0.67 	 0.62
0 	 25 	 0.62 	 0.62 	 0.71 	 0.67
0 	 26 	 0.57 	 0.58 	 0.67 	 0.62
0 	 27 	 0.57 	 0.58 	 0.67 	 0.62
0 	 28 	 0.55 	 0.54 	 0.65 	 0.59
0 	 2